In [1]:
import torch

from src import Detector, Siddon

In [2]:
# Make a test class
volume = torch.rand(4, 4, 4).abs()
siddon = Siddon(spacing=[1.0, 1.0, 1.0], center=[0.0, 0.0, 0.0], volume=volume)

## Single Ray

In [3]:
# Test the raytrace function
source = [-1.5, -1.0, -1.0]
target = [6.0, 6.0, 6.0]

source = torch.tensor(source, requires_grad=True)
target = torch.tensor(target, requires_grad=True)

radiologic_path_length = siddon.raytrace(source, target)
radiologic_path_length

tensor(0.2437, grad_fn=<AddBackward0>)

In [4]:
# Test backwards AD
radiologic_path_length.backward()

source.grad, target.grad

(tensor([-0.0312,  0.1022, -0.0340]), tensor([-0.0042,  0.0042, -0.0345]))

This implementation of the `Siddon` raytracing method is differentiable with respect to the `source` and `target` locations, the spatial parameters of our simulated X-ray setup.

In [5]:
# Timing experiment results on single-core CPU
%timeit siddon.raytrace(source, target)

350 µs ± 4.91 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Detector Plane

In [6]:
source = [-1.5, -1.0, -1.0]
target = [6.0, 6.0, 6.0]

detector = Detector(
    source=source,
    center=target,
    height=3,
    width=3,
    delx=0.1,
    dely=0.1,
)

rays = detector.make_xrays()

In [7]:
rays

[(tensor([-1.5000, -1.0000, -1.0000], requires_grad=True),
  tensor([6.0538, 6.1645, 5.7778], grad_fn=<AddBackward0>)),
 (tensor([-1.5000, -1.0000, -1.0000], requires_grad=True),
  tensor([6.0949, 6.0823, 5.8161], grad_fn=<AddBackward0>)),
 (tensor([-1.5000, -1.0000, -1.0000], requires_grad=True),
  tensor([6.1359, 6.0000, 5.8544], grad_fn=<AddBackward0>)),
 (tensor([-1.5000, -1.0000, -1.0000], requires_grad=True),
  tensor([5.9859, 6.1645, 5.8506], grad_fn=<AddBackward0>)),
 (tensor([-1.5000, -1.0000, -1.0000], requires_grad=True),
  tensor([6.0269, 6.0823, 5.8889], grad_fn=<AddBackward0>)),
 (tensor([-1.5000, -1.0000, -1.0000], requires_grad=True),
  tensor([6.0679, 6.0000, 5.9272], grad_fn=<AddBackward0>)),
 (tensor([-1.5000, -1.0000, -1.0000], requires_grad=True),
  tensor([5.9179, 6.1645, 5.9234], grad_fn=<AddBackward0>)),
 (tensor([-1.5000, -1.0000, -1.0000], requires_grad=True),
  tensor([5.9590, 6.0823, 5.9617], grad_fn=<AddBackward0>)),
 (tensor([-1.5000, -1.0000, -1.0000], re

In [8]:
drr = [siddon.raytrace(source, target) for source, target in rays]

In [9]:
torch.tensor(drr).reshape(3, 3)

tensor([[0.2521, 0.2502, 0.2483],
        [0.2497, 0.2479, 0.2460],
        [0.2474, 0.2456, 0.2437]])